## まとめ

このチュートリアルでは、微動アレイ観測解析の基礎から実践までを学びました：

1. **基礎理論**
   - 波動の基本概念（周波数、波長、位相速度）
   - 位相速度と群速度の違い
   - 表面波の分散性

2. **SPAC法**
   - 空間自己相関とベッセル関数
   - 観測点間距離と波長の関係
   - 位相速度の推定方法

3. **F-K法**
   - 時空間データの周波数-波数変換
   - ビームフォーミングによる波の検出
   - アレイ応答関数の重要性

4. **実データ解析**
   - データの前処理（トレンド除去、フィルタリング）
   - 品質管理（コヒーレンス）
   - 複数の解析手法の統合

5. **地下構造推定**
   - 分散曲線から速度構造への逆解析
   - モデルパラメータの最適化
   - 推定結果の評価

### 今後の学習のために

- より高度な逆解析手法（遺伝的アルゴリズム、MCMC法など）
- 高次モードの考慮
- 3次元構造の推定
- 実際の観測データの取り扱い

### 参考文献

- Aki, K. (1957): Space and time spectra of stationary stochastic waves
- Okada, H. (2003): The Microtremor Survey Method, SEG
- 物理探査学会 (2008): 物理探査ハンドブック

In [ ]:
def rayleigh_dispersion_curve(frequencies, layer_thickness, vs, vp_vs_ratio=1.73, density=2000):
    """
    レイリー波の理論分散曲線を計算（簡易版）
    
    Parameters:
    -----------
    frequencies : array
        周波数配列 [Hz]
    layer_thickness : array
        各層の層厚 [m]（最下層は半無限）
    vs : array
        各層のS波速度 [m/s]
    vp_vs_ratio : float
        Vp/Vs比
    density : float or array
        密度 [kg/m^3]
    
    Returns:
    --------
    phase_velocities : array
        位相速度 [m/s]
    """
    # 簡易的な分散曲線の計算（実際にはもっと複雑）
    # ここでは近似的な計算を行う
    
    n_layers = len(vs)
    phase_velocities = np.zeros(len(frequencies))
    
    # 各周波数での位相速度を計算
    for i, f in enumerate(frequencies):
        # 波長
        wavelength_approx = vs[0] / f  # 初期推定
        
        # 有効深度（波長の1/3程度）
        effective_depth = wavelength_approx / 3
        
        # 深度による重み付き平均速度
        depth_sum = 0
        weighted_vs = 0
        
        for j in range(n_layers):
            if j < n_layers - 1:
                layer_bottom = depth_sum + layer_thickness[j]
                layer_contrib = min(layer_thickness[j], 
                                  max(0, effective_depth - depth_sum))
            else:
                # 最下層（半無限）
                layer_contrib = max(0, effective_depth - depth_sum)
            
            if layer_contrib > 0:
                weight = np.exp(-depth_sum / effective_depth)
                weighted_vs += vs[j] * layer_contrib * weight
            
            if j < n_layers - 1:
                depth_sum += layer_thickness[j]
                if depth_sum > effective_depth:
                    break
        
        # レイリー波の位相速度（S波速度の約0.92倍）
        phase_velocities[i] = 0.92 * weighted_vs / effective_depth
    
    return phase_velocities

def plot_velocity_model_and_dispersion(layer_thickness, vs, frequencies, 
                                      observed_velocities=None):
    """
    速度構造モデルと分散曲線を表示
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # 速度構造モデル
    depths = np.concatenate([[0], np.cumsum(layer_thickness[:-1]), 
                            [np.sum(layer_thickness[:-1]) * 1.5]])
    
    # 階段状のプロット
    for i in range(len(vs)):
        if i < len(vs) - 1:
            ax1.plot([vs[i], vs[i]], [depths[i], depths[i+1]], 'b-', linewidth=2)
            if i < len(vs) - 2:
                ax1.plot([vs[i], vs[i+1]], [depths[i+1], depths[i+1]], 'b-', linewidth=2)
        else:
            # 最下層
            ax1.plot([vs[i], vs[i]], [depths[i], depths[i+1]], 'b-', linewidth=2)
    
    ax1.set_xlabel('S-wave Velocity [m/s]')
    ax1.set_ylabel('Depth [m]')
    ax1.set_title('Velocity Model')
    ax1.grid(True, alpha=0.3)
    ax1.invert_yaxis()
    ax1.set_xlim(0, max(vs) * 1.2)
    
    # 分散曲線
    theoretical_velocities = rayleigh_dispersion_curve(frequencies, 
                                                      layer_thickness, vs)
    
    ax2.plot(frequencies, theoretical_velocities, 'b-', linewidth=2, 
             label='Theoretical')
    
    if observed_velocities is not None:
        ax2.plot(frequencies, observed_velocities, 'ro', markersize=6, 
                alpha=0.7, label='Observed')
    
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Phase Velocity [m/s]')
    ax2.set_title('Dispersion Curve')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xscale('log')
    
    plt.tight_layout()
    plt.show()
    
    return theoretical_velocities

# インタラクティブな速度構造モデリング
@widgets.interact(
    h1=(5, 20, 1),
    h2=(10, 40, 2),
    vs1=(100, 300, 10),
    vs2=(200, 500, 20),
    vs3=(300, 800, 50)
)
def interactive_velocity_model(h1=10, h2=20, vs1=150, vs2=300, vs3=500):
    """
    層厚と速度を変えて分散曲線の変化を観察
    """
    # モデルパラメータ
    layer_thickness = np.array([h1, h2, np.inf])
    vs = np.array([vs1, vs2, vs3])
    
    # 周波数範囲
    frequencies = np.logspace(-0.5, 1.2, 50)
    
    # 「観測」分散曲線（真のモデルから生成）
    true_thickness = np.array([8, 25, np.inf])
    true_vs = np.array([180, 350, 600])
    observed_velocities = rayleigh_dispersion_curve(frequencies, 
                                                   true_thickness, true_vs)
    
    # プロット
    theoretical_velocities = plot_velocity_model_and_dispersion(
        layer_thickness, vs, frequencies, observed_velocities
    )
    
    # 残差の計算
    rms_error = np.sqrt(np.mean((theoretical_velocities - observed_velocities)**2))
    print(f"\nRMS誤差: {rms_error:.1f} m/s")
    print(f"相対誤差: {rms_error/np.mean(observed_velocities)*100:.1f}%")

## 6. 地下構造推定

### 6.1 分散曲線から速度構造へ

分散曲線（位相速度の周波数依存性）から地下のS波速度構造を推定します。
これは**逆問題**であり、以下の手順で解きます：

1. **初期モデルの設定**（層数、層厚、速度）
2. **理論分散曲線の計算**（順問題）
3. **観測値との残差計算**
4. **モデルの更新**（最適化）
5. **収束判定**

In [ ]:
class MicrotremorArray:
    """
    微動アレイ解析用クラス
    """
    def __init__(self, data, coordinates, fs):
        """
        Parameters:
        -----------
        data : array (n_stations, n_samples)
            各観測点の時系列データ
        coordinates : array (n_stations, 2)
            観測点座標 (x, y) [m]
        fs : float
            サンプリング周波数 [Hz]
        """
        self.data = data
        self.coordinates = coordinates
        self.fs = fs
        self.n_stations, self.n_samples = data.shape
        
        # 観測点間距離の計算
        self.distances = self._calculate_distances()
        
    def _calculate_distances(self):
        """観測点間距離行列の計算"""
        n = self.n_stations
        distances = np.zeros((n, n))
        for i in range(n):
            for j in range(i+1, n):
                dist = np.linalg.norm(self.coordinates[i] - self.coordinates[j])
                distances[i, j] = dist
                distances[j, i] = dist
        return distances
    
    def preprocess(self, detrend_type='linear', f_low=0.1, f_high=20):
        """
        データの前処理
        
        Parameters:
        -----------
        detrend_type : str
            トレンド除去の方法 ('linear', 'constant')
        f_low, f_high : float
            バンドパスフィルタの周波数範囲 [Hz]
        """
        processed_data = np.zeros_like(self.data)
        
        for i in range(self.n_stations):
            # トレンド除去
            data_detrend = signal.detrend(self.data[i], type=detrend_type)
            
            # バンドパスフィルタ
            sos = signal.butter(4, [f_low, f_high], btype='band', 
                               fs=self.fs, output='sos')
            data_filtered = signal.sosfilt(sos, data_detrend)
            
            # テーパー処理（端部の影響を低減）
            taper = signal.windows.tukey(len(data_filtered), alpha=0.1)
            processed_data[i] = data_filtered * taper
        
        self.processed_data = processed_data
        return processed_data
    
    def compute_spac_all_pairs(self, nperseg=2048, noverlap=1024):
        """
        全観測点ペアのSPAC係数を計算
        """
        # 使用するデータ
        data_to_use = self.processed_data if hasattr(self, 'processed_data') else self.data
        
        # 結果を格納する辞書
        spac_results = {}
        
        # 全ペアについて計算
        for i in range(self.n_stations):
            for j in range(i+1, self.n_stations):
                # SPAC係数の計算
                f, spac_coef, coherence = calculate_spac_coefficient(
                    data_to_use[i], data_to_use[j], self.fs, 
                    nperseg=nperseg, noverlap=noverlap
                )
                
                # 結果を保存
                distance = self.distances[i, j]
                spac_results[(i, j)] = {
                    'distance': distance,
                    'frequencies': f,
                    'spac_coef': spac_coef,
                    'coherence': coherence
                }
        
        self.spac_results = spac_results
        return spac_results
    
    def plot_spac_by_distance(self, f_min=0.1, f_max=10):
        """
        距離ごとのSPAC係数をプロット
        """
        if not hasattr(self, 'spac_results'):
            raise ValueError("先にcompute_spac_all_pairs()を実行してください")
        
        # 距離でグループ化
        distance_groups = {}
        for pair, result in self.spac_results.items():
            dist = result['distance']
            if dist not in distance_groups:
                distance_groups[dist] = []
            distance_groups[dist].append(result)
        
        # プロット
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        axes = axes.ravel()
        
        distances_sorted = sorted(distance_groups.keys())
        
        for idx, dist in enumerate(distances_sorted[:4]):
            ax = axes[idx]
            
            # この距離での全ペアの平均
            spac_sum = None
            count = 0
            
            for result in distance_groups[dist]:
                f = result['frequencies']
                mask = (f >= f_min) & (f <= f_max)
                
                if spac_sum is None:
                    spac_sum = result['spac_coef'][mask]
                    f_plot = f[mask]
                else:
                    spac_sum += result['spac_coef'][mask]
                count += 1
            
            spac_mean = spac_sum / count
            
            # プロット
            ax.plot(f_plot, spac_mean, 'ko', markersize=4, alpha=0.5)
            ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
            
            # 理論曲線をフィット
            try:
                f_fit, c_fit, spac_fit = estimate_phase_velocity_spac(
                    f_plot, spac_mean, dist, f_min=f_min, f_max=f_max
                )
                if c_fit is not None:
                    ax.plot(f_fit, spac_fit, 'r-', linewidth=2,
                           label=f'c = {c_fit:.1f} m/s')
            except:
                pass
            
            ax.set_xlabel('Frequency [Hz]')
            ax.set_ylabel('SPAC Coefficient')
            ax.set_title(f'Distance = {dist:.1f} m')
            ax.legend()
            ax.grid(True, alpha=0.3)
            ax.set_ylim(-0.5, 1.1)
        
        plt.tight_layout()
        plt.show()

# 実データ解析のデモンストレーション
def demo_real_data_analysis():
    # アレイ配置（実際の観測を模擬）
    # 二重三角形アレイ
    inner_radius = 10  # 内側の半径
    outer_radius = 30  # 外側の半径
    
    angles = np.linspace(0, 2*np.pi, 4)[:-1]  # 三角形の頂点
    
    coordinates = [[0, 0]]  # 中心点
    # 内側の三角形
    for angle in angles:
        coordinates.append([inner_radius * np.cos(angle), 
                           inner_radius * np.sin(angle)])
    # 外側の三角形
    for angle in angles:
        coordinates.append([outer_radius * np.cos(angle), 
                           outer_radius * np.sin(angle)])
    
    coordinates = np.array(coordinates)
    n_stations = len(coordinates)
    
    # 実データを模擬（複数の表面波モード）
    fs = 50  # サンプリング周波数
    duration = 300  # 5分間の観測
    t = np.arange(0, duration, 1/fs)
    n_samples = len(t)
    
    # 複数の平面波の重ね合わせ
    wave_params = [
        {'f': 0.5, 'c': 500, 'theta': np.pi/6, 'amp': 0.5},
        {'f': 1.0, 'c': 400, 'theta': np.pi/4, 'amp': 1.0},
        {'f': 2.0, 'c': 300, 'theta': np.pi/3, 'amp': 0.8},
        {'f': 5.0, 'c': 200, 'theta': np.pi/2, 'amp': 0.4},
    ]
    
    data = np.zeros((n_stations, n_samples))
    
    for params in wave_params:
        f = params['f']
        c = params['c']
        theta = params['theta']
        amp = params['amp']
        
        k = 2 * np.pi * f / c
        kx = k * np.cos(theta)
        ky = k * np.sin(theta)
        
        for i_sta in range(n_stations):
            x, y = coordinates[i_sta]
            phase = kx * x + ky * y - 2 * np.pi * f * t + np.random.rand() * 2 * np.pi
            data[i_sta] += amp * np.sin(phase)
    
    # ノイズを追加
    noise_level = 0.3
    data += noise_level * np.random.randn(n_stations, n_samples)
    
    # MicrotremorArrayオブジェクトの作成
    array = MicrotremorArray(data, coordinates, fs)
    
    # 前処理
    array.preprocess(f_low=0.2, f_high=10)
    
    # SPAC解析
    array.compute_spac_all_pairs(nperseg=2048, noverlap=1536)
    
    # 結果の表示
    print("アレイ解析の概要：")
    print(f"観測点数: {n_stations}")
    print(f"サンプリング周波数: {fs} Hz")
    print(f"観測時間: {duration} 秒")
    print(f"\n観測点間距離:")
    unique_distances = np.unique(array.distances[array.distances > 0])
    for d in unique_distances:
        count = np.sum(array.distances == d) // 2  # 各ペアは2回カウントされる
        print(f"  {d:.1f} m: {count} ペア")
    
    # SPAC係数のプロット
    array.plot_spac_by_distance()
    
    return array

# 実行
array = demo_real_data_analysis()

## 5. 実データ解析

### 5.1 データの前処理

実際の微動データ解析では、以下の前処理が重要です：

1. **機器応答の補正**
2. **トレンド除去**
3. **バンドパスフィルタ**
4. **過渡的ノイズの除去**

In [ ]:
def fk_analysis(data_array, coordinates, fs, f_target, window_length=10.0):
    """
    F-K解析の実装
    
    Parameters:
    -----------
    data_array : array (n_stations, n_samples)
        各観測点の時系列データ
    coordinates : array (n_stations, 2)
        観測点の座標 (x, y) [m]
    fs : float
        サンプリング周波数 [Hz]
    f_target : float
        解析対象の周波数 [Hz]
    window_length : float
        時間窓長 [s]
    
    Returns:
    --------
    kx, ky : array
        波数グリッド
    fk_spectrum : array
        F-Kスペクトル
    c_estimated : float
        推定された位相速度 [m/s]
    theta : float
        到来方向 [rad]
    """
    n_stations, n_samples = data_array.shape
    
    # 時間窓の設定
    window_samples = int(window_length * fs)
    n_windows = n_samples // window_samples
    
    # 波数グリッドの設定
    k_max = 0.1  # 最大波数 [1/m]
    dk = 0.001   # 波数分解能
    kx = np.arange(-k_max, k_max, dk)
    ky = np.arange(-k_max, k_max, dk)
    KX, KY = np.meshgrid(kx, ky)
    
    # F-Kスペクトルの初期化
    fk_spectrum = np.zeros((len(ky), len(kx)))
    
    # 各時間窓で解析
    for i_win in range(n_windows):
        start_idx = i_win * window_samples
        end_idx = start_idx + window_samples
        
        # 各観測点でフーリエ変換
        fft_data = np.zeros(n_stations, dtype=complex)
        for i_sta in range(n_stations):
            data_segment = data_array[i_sta, start_idx:end_idx]
            # 窓関数の適用
            data_segment *= signal.windows.hann(window_samples)
            # FFT
            fft_result = np.fft.fft(data_segment)
            freq = np.fft.fftfreq(window_samples, 1/fs)
            # 対象周波数成分の抽出
            f_idx = np.argmin(np.abs(freq - f_target))
            fft_data[i_sta] = fft_result[f_idx]
        
        # ビームフォーミング
        for i_ky, ky_val in enumerate(ky):
            for i_kx, kx_val in enumerate(kx):
                # 平面波の位相シフト
                phase_shift = np.exp(1j * 2 * np.pi * 
                                   (kx_val * coordinates[:, 0] + 
                                    ky_val * coordinates[:, 1]))
                # ビームパワー
                beam = np.sum(fft_data * phase_shift)
                fk_spectrum[i_ky, i_kx] += np.abs(beam)**2
    
    # 正規化
    fk_spectrum /= (n_windows * n_stations**2)
    
    # ピーク検出
    max_idx = np.unravel_index(np.argmax(fk_spectrum), fk_spectrum.shape)
    kx_max = kx[max_idx[1]]
    ky_max = ky[max_idx[0]]
    k_max = np.sqrt(kx_max**2 + ky_max**2)
    
    # 位相速度と到来方向
    if k_max > 0:
        c_estimated = 2 * np.pi * f_target / k_max
        theta = np.arctan2(ky_max, kx_max)
    else:
        c_estimated = np.inf
        theta = 0
    
    return kx, ky, fk_spectrum, c_estimated, theta

# デモンストレーション：円形アレイでのF-K解析
def demo_fk_analysis():
    # アレイ配置（円形アレイ）
    n_stations = 7  # 中心 + 周辺6点
    radius = 30  # アレイ半径 [m]
    
    coordinates = np.zeros((n_stations, 2))
    coordinates[0, :] = [0, 0]  # 中心点
    for i in range(1, n_stations):
        angle = 2 * np.pi * (i-1) / (n_stations-1)
        coordinates[i, 0] = radius * np.cos(angle)
        coordinates[i, 1] = radius * np.sin(angle)
    
    # 合成波動場の生成
    fs = 100  # サンプリング周波数
    duration = 30  # 観測時間
    t = np.arange(0, duration, 1/fs)
    n_samples = len(t)
    
    # 平面波のパラメータ
    f_wave = 2.0  # 周波数 [Hz]
    c_true = 300  # 真の位相速度 [m/s]
    theta_true = np.pi/4  # 到来方向 [rad]
    k_true = 2 * np.pi * f_wave / c_true
    kx_true = k_true * np.cos(theta_true)
    ky_true = k_true * np.sin(theta_true)
    
    # 各観測点での信号生成
    data_array = np.zeros((n_stations, n_samples))
    for i_sta in range(n_stations):
        x, y = coordinates[i_sta]
        phase = kx_true * x + ky_true * y - 2 * np.pi * f_wave * t
        data_array[i_sta, :] = np.sin(phase) + 0.3 * np.random.randn(n_samples)
    
    # F-K解析
    kx, ky, fk_spectrum, c_estimated, theta_estimated = fk_analysis(
        data_array, coordinates, fs, f_wave
    )
    
    # 結果の可視化
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # アレイ配置
    ax = axes[0, 0]
    ax.scatter(coordinates[0, 0], coordinates[0, 1], s=200, c='red', 
               marker='^', label='Center')
    ax.scatter(coordinates[1:, 0], coordinates[1:, 1], s=200, c='blue', 
               marker='v', label='Outer')
    # 真の到来方向
    arrow_len = radius * 1.5
    ax.arrow(0, 0, arrow_len*np.cos(theta_true), arrow_len*np.sin(theta_true),
             head_width=5, head_length=5, fc='green', ec='green', 
             linewidth=2, label='True direction')
    ax.set_xlim(-50, 50)
    ax.set_ylim(-50, 50)
    ax.set_aspect('equal')
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_title('Array Configuration')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 時系列データ
    ax = axes[0, 1]
    time_plot = t[:500]
    for i_sta in range(min(4, n_stations)):
        ax.plot(time_plot, data_array[i_sta, :500] + i_sta*3, 
                label=f'Station {i_sta}')
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Amplitude (offset)')
    ax.set_title('Waveforms')
    ax.grid(True, alpha=0.3)
    
    # F-Kスペクトル
    ax = axes[1, 0]
    KX, KY = np.meshgrid(kx, ky)
    im = ax.contourf(KX, KY, fk_spectrum, levels=20, cmap='hot')
    ax.contour(KX, KY, fk_spectrum, levels=10, colors='white', 
               linewidths=0.5, alpha=0.5)
    
    # 真の波数をマーク
    ax.plot(kx_true, ky_true, 'b*', markersize=15, label='True')
    # 推定された波数をマーク
    k_est = 2 * np.pi * f_wave / c_estimated
    kx_est = k_est * np.cos(theta_estimated)
    ky_est = k_est * np.sin(theta_estimated)
    ax.plot(kx_est, ky_est, 'g^', markersize=12, label='Estimated')
    
    ax.set_xlabel('$k_x$ [1/m]')
    ax.set_ylabel('$k_y$ [1/m]')
    ax.set_title(f'F-K Spectrum (f = {f_wave} Hz)')
    ax.set_aspect('equal')
    ax.legend()
    plt.colorbar(im, ax=ax, label='Power')
    
    # スローネス表示
    ax = axes[1, 1]
    # スローネスに変換
    sx = kx / (2 * np.pi * f_wave)
    sy = ky / (2 * np.pi * f_wave)
    SX, SY = np.meshgrid(sx, sy)
    
    im = ax.contourf(SX * 1000, SY * 1000, fk_spectrum, levels=20, cmap='hot')
    
    # 速度円を描画
    velocities = [200, 300, 500, 1000]
    for v in velocities:
        s = 1000 / v  # スローネス [s/km]
        circle = plt.Circle((0, 0), s, fill=False, color='white', 
                           linestyle='--', alpha=0.5)
        ax.add_artist(circle)
        ax.text(0, s, f'{v} m/s', color='white', fontsize=8, 
                ha='center', va='bottom')
    
    ax.set_xlabel('$s_x$ [s/km]')
    ax.set_ylabel('$s_y$ [s/km]')
    ax.set_title('Slowness Domain')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_aspect('equal')
    plt.colorbar(im, ax=ax, label='Power')
    
    plt.tight_layout()
    plt.show()
    
    # 結果の表示
    print(f"\n解析結果：")
    print(f"真の位相速度: {c_true:.1f} m/s")
    print(f"推定位相速度: {c_estimated:.1f} m/s")
    print(f"誤差: {abs(c_estimated - c_true)/c_true*100:.1f}%")
    print(f"\n真の到来方向: {np.degrees(theta_true):.1f}°")
    print(f"推定到来方向: {np.degrees(theta_estimated):.1f}°")

demo_fk_analysis()

## 4. F-K法の原理と実装

### 4.1 周波数-波数法（F-K法）の基本概念

F-K法は、時空間データを周波数-波数領域に変換し、波の伝播特性を解析する手法です。

**基本原理：**
1. 時間方向のフーリエ変換 → 各周波数成分を抽出
2. 空間方向のフーリエ変換 → 波数スペクトルを計算
3. F-Kスペクトルのピーク → 卓越する波の波数と伝播方向
4. 位相速度 = 周波数 / 波数

$$c = \frac{\omega}{|\mathbf{k}|} = \frac{2\pi f}{\sqrt{k_x^2 + k_y^2}}$$

In [ ]:
def calculate_spac_coefficient(data1, data2, fs, nperseg=1024, noverlap=512):
    """
    2つの観測点間のSPAC係数を計算
    
    Parameters:
    -----------
    data1, data2 : array-like
        各観測点の時系列データ
    fs : float
        サンプリング周波数 [Hz]
    nperseg : int
        セグメント長
    noverlap : int
        オーバーラップ長
    
    Returns:
    --------
    frequencies : array
        周波数配列
    spac_coef : array
        SPAC係数
    coherence : array
        コヒーレンス（品質指標）
    """
    # クロススペクトル密度
    f, Pxy = signal.csd(data1, data2, fs=fs, nperseg=nperseg, noverlap=noverlap)
    
    # パワースペクトル密度
    _, Pxx = signal.welch(data1, fs=fs, nperseg=nperseg, noverlap=noverlap)
    _, Pyy = signal.welch(data2, fs=fs, nperseg=nperseg, noverlap=noverlap)
    
    # SPAC係数（実部のみ使用）
    spac_coef = np.real(Pxy) / np.sqrt(Pxx * Pyy)
    
    # コヒーレンス
    coherence = np.abs(Pxy)**2 / (Pxx * Pyy)
    
    return f, spac_coef, coherence

def estimate_phase_velocity_spac(frequencies, spac_coef, distance, f_min=0.1, f_max=10):
    """
    SPAC係数から位相速度を推定
    """
    # 解析範囲の選択
    mask = (frequencies >= f_min) & (frequencies <= f_max)
    f_analysis = frequencies[mask]
    spac_analysis = spac_coef[mask]
    
    # ベッセル関数フィッティング
    def bessel_func(f, c):
        return special.j0(2 * np.pi * distance * f / c)
    
    # 初期値の設定（複数試行）
    c_init_values = [100, 200, 300, 500]
    best_params = None
    best_error = np.inf
    
    for c_init in c_init_values:
        try:
            params, _ = curve_fit(bessel_func, f_analysis, spac_analysis, 
                                  p0=[c_init], bounds=(50, 2000))
            error = np.mean((spac_analysis - bessel_func(f_analysis, params[0]))**2)
            if error < best_error:
                best_error = error
                best_params = params
        except:
            continue
    
    if best_params is not None:
        c_estimated = best_params[0]
        spac_fitted = bessel_func(f_analysis, c_estimated)
        return f_analysis, c_estimated, spac_fitted
    else:
        return None, None, None

# デモンストレーション用の合成データ生成
def generate_synthetic_microtremor(duration=60, fs=100, c_true=250, f_dominant=2.0):
    """
    合成微動データの生成
    """
    t = np.arange(0, duration, 1/fs)
    n_samples = len(t)
    
    # 複数の周波数成分を持つ信号
    frequencies = np.array([0.5, 1.0, 2.0, 4.0, 8.0])
    amplitudes = np.array([0.3, 0.5, 1.0, 0.7, 0.4])
    
    # 観測点1（原点）での信号
    signal1 = np.zeros(n_samples)
    for f, a in zip(frequencies, amplitudes):
        phase = np.random.rand() * 2 * np.pi
        signal1 += a * np.sin(2 * np.pi * f * t + phase)
    
    # ノイズを追加
    signal1 += 0.2 * np.random.randn(n_samples)
    
    # 観測点2での信号（距離50mでの遅延を考慮）
    distance = 50
    signal2 = np.zeros(n_samples)
    for f, a in zip(frequencies, amplitudes):
        k = 2 * np.pi * f / c_true
        phase_shift = k * distance
        phase = np.random.rand() * 2 * np.pi
        signal2 += a * np.sin(2 * np.pi * f * t + phase - phase_shift)
    
    signal2 += 0.2 * np.random.randn(n_samples)
    
    return t, signal1, signal2, distance

# 合成データの生成と解析
t, data1, data2, distance = generate_synthetic_microtremor()
fs = 100  # サンプリング周波数

# SPAC係数の計算
f, spac_coef, coherence = calculate_spac_coefficient(data1, data2, fs)

# 位相速度の推定
f_analysis, c_estimated, spac_fitted = estimate_phase_velocity_spac(f, spac_coef, distance)

# 結果の表示
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# 時系列データ
ax1.plot(t[:500], data1[:500], 'b-', alpha=0.7, label='Station 1')
ax1.plot(t[:500], data2[:500], 'r-', alpha=0.7, label='Station 2')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Amplitude')
ax1.set_title('Synthetic Microtremor Data')
ax1.legend()
ax1.grid(True, alpha=0.3)

# パワースペクトル
f_psd, psd1 = signal.welch(data1, fs=fs)
_, psd2 = signal.welch(data2, fs=fs)
ax2.semilogy(f_psd, psd1, 'b-', label='Station 1')
ax2.semilogy(f_psd, psd2, 'r-', label='Station 2')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('PSD')
ax2.set_title('Power Spectral Density')
ax2.legend()
ax2.grid(True, alpha=0.3)

# SPAC係数とフィッティング
ax3.plot(f, spac_coef, 'ko', markersize=4, alpha=0.5, label='Observed')
if c_estimated is not None:
    ax3.plot(f_analysis, spac_fitted, 'r-', linewidth=2, 
             label=f'Fitted (c = {c_estimated:.1f} m/s)')
ax3.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax3.set_xlabel('Frequency [Hz]')
ax3.set_ylabel('SPAC Coefficient')
ax3.set_title(f'SPAC Analysis (r = {distance} m)')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, 10)
ax3.set_ylim(-0.5, 1.1)

# コヒーレンス
ax4.plot(f, coherence, 'g-', linewidth=2)
ax4.axhline(y=0.8, color='r', linestyle='--', alpha=0.5, label='Threshold (0.8)')
ax4.set_xlabel('Frequency [Hz]')
ax4.set_ylabel('Coherence')
ax4.set_title('Coherence (Quality Indicator)')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, 10)
ax4.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

if c_estimated is not None:
    print(f"\nEstimated phase velocity: {c_estimated:.1f} m/s")
    print(f"True phase velocity: 250.0 m/s")
    print(f"Error: {abs(c_estimated - 250)/250*100:.1f}%")

### 3.2 SPAC法の実装

実際のデータ解析では、以下の手順で空間自己相関係数を計算します：

1. **時系列データのフーリエ変換**
2. **クロススペクトルとパワースペクトルの計算**
3. **空間自己相関係数の算出**
4. **ベッセル関数フィッティングによる位相速度推定**

In [ ]:
# インタラクティブなSPAC解析
@widgets.interact(r=(10, 100, 10), c=(100, 500, 50))
def interactive_spac(r=50, c=200):
    """
    観測点間距離と位相速度を変えて空間相関を観察
    """
    frequencies = np.linspace(0.1, 20, 200)
    correlation = special.j0(2 * np.pi * r * frequencies / c)
    
    # 理論的な波長
    wavelengths = c / frequencies
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 周波数に対する相関
    ax1.plot(frequencies, correlation, 'b-', linewidth=2)
    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax1.fill_between(frequencies, 0, correlation, where=(correlation > 0), 
                     alpha=0.3, color='blue')
    ax1.fill_between(frequencies, 0, correlation, where=(correlation < 0), 
                     alpha=0.3, color='red')
    
    # 最初のゼロクロス点を見つける
    zero_crossings = np.where(np.diff(np.sign(correlation)))[0]
    if len(zero_crossings) > 0:
        f_zero = frequencies[zero_crossings[0]]
        ax1.axvline(x=f_zero, color='green', linestyle=':', linewidth=2)
        ax1.text(f_zero, 0.5, f'f = {f_zero:.2f} Hz\n$\lambda$ = {c/f_zero:.1f} m', 
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
    
    ax1.set_xlabel('Frequency [Hz]')
    ax1.set_ylabel('Spatial Correlation')
    ax1.set_title(f'SPAC Coefficient (r = {r} m, c = {c} m/s)')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(-0.5, 1.1)
    
    # r/λ に対する相関
    r_over_lambda = r / wavelengths
    ax2.plot(r_over_lambda, correlation, 'r-', linewidth=2)
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax2.axvline(x=0.38, color='green', linestyle=':', alpha=0.7, 
                label='First zero ($r/\lambda \\approx 0.38$)')
    
    ax2.set_xlabel('$r/\lambda$')
    ax2.set_ylabel('Spatial Correlation')
    ax2.set_title('Normalized Distance')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_xlim(0, 2)
    ax2.set_ylim(-0.5, 1.1)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# ベッセル関数の性質を理解する
def plot_bessel_function():
    x = np.linspace(0, 20, 1000)
    j0 = special.j0(x)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # ベッセル関数のプロット
    ax1.plot(x, j0, 'b-', linewidth=2, label='$J_0(x)$')
    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax1.axhline(y=1, color='r', linestyle=':', alpha=0.3)
    ax1.axhline(y=-1, color='r', linestyle=':', alpha=0.3)
    
    # ゼロ点をマーク
    zero_points = [2.405, 5.520, 8.654, 11.792, 14.931]
    for i, zp in enumerate(zero_points[:3]):
        ax1.axvline(x=zp, color='red', linestyle=':', alpha=0.5)
        ax1.text(zp, 0.1, f'{zp:.3f}', ha='center', fontsize=10, color='red')
    
    ax1.set_xlim(0, 15)
    ax1.set_ylim(-0.5, 1.2)
    ax1.set_xlabel('x')
    ax1.set_ylabel('$J_0(x)$')
    ax1.set_title('Bessel Function of the First Kind (Order 0)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # 空間相関の例
    r = 50  # 観測点間距離 [m]
    frequencies = np.linspace(0.1, 10, 100)
    c = 200  # 位相速度 [m/s]
    
    correlation = special.j0(2 * np.pi * r * frequencies / c)
    
    ax2.plot(frequencies, correlation, 'b-', linewidth=2)
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax2.fill_between(frequencies, 0, correlation, where=(correlation > 0), 
                     alpha=0.3, color='blue', label='Positive correlation')
    ax2.fill_between(frequencies, 0, correlation, where=(correlation < 0), 
                     alpha=0.3, color='red', label='Negative correlation')
    
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Spatial Correlation')
    ax2.set_title(f'Spatial Autocorrelation (r = {r} m, c = {c} m/s)')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_bessel_function()

## 3. SPAC法の原理と実装

### 3.1 空間自己相関法（SPAC）の基本概念

SPAC法は、2つの観測点間の相関から波の波長を推定する手法です。

**基本的なアイデア：**
- 観測点間距離が波長の整数倍 → 相関係数 = 1
- 観測点間距離が波長の半整数倍 → 相関係数 = -1
- 等方的な波動場では、相関係数は**ベッセル関数**に従う

$$\rho(r, f) = J_0\left(\frac{2\pi rf}{c(f)}\right)$$

ここで：
- $\rho(r, f)$：空間自己相関係数
- $J_0$：第1種0次ベッセル関数
- $r$：観測点間距離
- $f$：周波数
- $c(f)$：位相速度

In [ ]:
# 位相速度と群速度の違いを可視化
def plot_phase_group_velocity():
    # 2つの周波数成分の重ね合わせ
    x = np.linspace(0, 100, 1000)
    t_values = np.linspace(0, 2, 5)
    
    # 基本パラメータ
    k = 0.5  # 中心波数
    dk = 0.05  # 波数の差
    omega = 2.0  # 中心角周波数
    domega = 0.2  # 角周波数の差
    
    fig, axes = plt.subplots(len(t_values), 1, figsize=(12, 10), sharex=True)
    
    for i, t in enumerate(t_values):
        # 包絡線（群速度で移動）
        envelope = 2 * np.cos(dk * x - domega * t)
        # 搬送波（位相速度で移動）
        carrier = np.sin(k * x - omega * t)
        # 合成波
        wave = envelope * carrier
        
        ax = axes[i]
        ax.plot(x, wave, 'b-', linewidth=1.5, label='Wave packet')
        ax.plot(x, envelope, 'r--', linewidth=2, label='Envelope (Group velocity)')
        ax.plot(x, -envelope, 'r--', linewidth=2)
        
        # 位相速度と群速度の位置を示す
        phase_pos = (omega / k) * t
        group_pos = (domega / dk) * t
        
        if phase_pos < 100:
            ax.axvline(x=phase_pos, color='green', linestyle=':', alpha=0.7, label='Phase position')
        if group_pos < 100:
            ax.axvline(x=group_pos, color='red', linestyle=':', alpha=0.7, label='Group position')
        
        ax.set_ylabel('Amplitude')
        ax.set_title(f't = {t:.1f} s')
        ax.set_ylim(-3, 3)
        ax.grid(True, alpha=0.3)
        
        if i == 0:
            ax.legend(loc='upper right')
    
    axes[-1].set_xlabel('Distance [m]')
    fig.suptitle('Phase Velocity vs Group Velocity', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # 速度の表示
    c_phase = omega / k
    c_group = domega / dk
    print(f"Phase velocity: {c_phase:.2f} m/s")
    print(f"Group velocity: {c_group:.2f} m/s")

plot_phase_group_velocity()

### 2.2 位相速度と群速度

分散性媒質では、異なる周波数の波が異なる速度で伝播します。

- **位相速度** $c$：単一周波数成分の位相が進む速度
- **群速度** $U$：波束（エネルギー）が伝播する速度

$$U = \frac{d\omega}{dk} = c - \lambda\frac{dc}{d\lambda}$$

In [ ]:
# インタラクティブな波動シミュレーション
def wave_animation(A=1.0, f=1.0, c=100.0, phi=0.0):
    """
    波動の伝播をアニメーション表示
    A: 振幅, f: 周波数[Hz], c: 位相速度[m/s], phi: 初期位相[rad]
    """
    # パラメータ計算
    omega = 2 * np.pi * f
    k = omega / c
    wavelength = c / f
    
    # 空間と時間の設定
    x = np.linspace(0, 200, 1000)
    t_max = 2.0
    dt = 0.05
    times = np.arange(0, t_max, dt)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # 初期設定
    line1, = ax1.plot([], [], 'b-', linewidth=2)
    point1, = ax1.plot([], [], 'ro', markersize=10)
    point2, = ax1.plot([], [], 'go', markersize=10)
    
    ax1.set_xlim(0, 200)
    ax1.set_ylim(-2*A, 2*A)
    ax1.set_xlabel('Distance [m]')
    ax1.set_ylabel('Amplitude')
    ax1.set_title('Wave Propagation')
    ax1.grid(True, alpha=0.3)
    
    # パラメータ表示
    info_text = f'f = {f:.1f} Hz, $\lambda$ = {wavelength:.1f} m, c = {c:.1f} m/s'
    ax1.text(0.02, 0.95, info_text, transform=ax1.transAxes, 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # 2点での時系列
    x1, x2 = 50, 100  # 観測点の位置
    time_series1 = []
    time_series2 = []
    time_points = []
    
    line2, = ax2.plot([], [], 'r-', label=f'x = {x1} m')
    line3, = ax2.plot([], [], 'g-', label=f'x = {x2} m')
    ax2.set_xlim(0, t_max)
    ax2.set_ylim(-2*A, 2*A)
    ax2.set_xlabel('Time [s]')
    ax2.set_ylabel('Amplitude')
    ax2.set_title('Time Series at Two Points')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    def animate(frame):
        t = times[frame]
        u = A * np.sin(k * x - omega * t + phi)
        u1 = A * np.sin(k * x1 - omega * t + phi)
        u2 = A * np.sin(k * x2 - omega * t + phi)
        
        line1.set_data(x, u)
        point1.set_data([x1], [u1])
        point2.set_data([x2], [u2])
        
        time_series1.append(u1)
        time_series2.append(u2)
        time_points.append(t)
        
        if len(time_points) > 0:
            line2.set_data(time_points, time_series1)
            line3.set_data(time_points, time_series2)
        
        return line1, point1, point2, line2, line3
    
    anim = animation.FuncAnimation(fig, animate, frames=len(times), 
                                   interval=50, blit=True)
    plt.tight_layout()
    return anim

# ウィジェットの作成
amplitude_slider = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Amplitude:')
frequency_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Frequency [Hz]:')
velocity_slider = widgets.FloatSlider(value=100.0, min=50.0, max=500.0, step=10.0, description='Velocity [m/s]:')
phase_slider = widgets.FloatSlider(value=0.0, min=0.0, max=2*np.pi, step=0.1, description='Phase [rad]:')

# 静的な波形表示（アニメーションの代わり）
@widgets.interact(A=amplitude_slider, f=frequency_slider, c=velocity_slider, phi=phase_slider)
def plot_wave(A=1.0, f=1.0, c=100.0, phi=0.0):
    omega = 2 * np.pi * f
    k = omega / c
    wavelength = c / f
    
    x = np.linspace(0, 200, 1000)
    t = 0  # 時刻t=0での波形
    u = A * np.sin(k * x - omega * t + phi)
    
    plt.figure(figsize=(12, 5))
    plt.plot(x, u, 'b-', linewidth=2)
    plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    # 波長を表示
    x_start = 50
    plt.plot([x_start, x_start + wavelength], [-A*1.3, -A*1.3], 'r-', linewidth=3)
    plt.plot([x_start, x_start], [-A*1.4, -A*1.2], 'r-', linewidth=2)
    plt.plot([x_start + wavelength, x_start + wavelength], [-A*1.4, -A*1.2], 'r-', linewidth=2)
    plt.text(x_start + wavelength/2, -A*1.5, f'$\lambda$ = {wavelength:.1f} m', 
             ha='center', fontsize=12, color='red')
    
    plt.xlim(0, 200)
    plt.ylim(-2*A, 2*A)
    plt.xlabel('Distance [m]')
    plt.ylabel('Amplitude')
    plt.title(f'Wave at t=0: f={f:.1f} Hz, c={c:.1f} m/s')
    plt.grid(True, alpha=0.3)
    plt.show()

## 2. 波動の基礎理論

### 2.1 単純な正弦波

位置 $x$ と時刻 $t$ における波の変位は次の式で表されます：

$$u(x, t) = A \sin(kx - \omega t + \phi)$$

ここで：
- $A$：振幅
- $k = 2\pi/\lambda$：波数
- $\omega = 2\pi f$：角周波数
- $\phi$：初期位相
- $\lambda$：波長
- $f$：周波数

**位相速度**は次の式で計算されます：
$$c = \frac{\omega}{k} = f \times \lambda$$

In [ ]:
# 微動アレイ観測の概念図
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# アレイ配置の例
angles = np.linspace(0, 2*np.pi, 7)
r = 50  # アレイ半径 [m]
x_array = r * np.cos(angles[:-1])
y_array = r * np.sin(angles[:-1])

ax1.scatter(0, 0, s=200, c='red', marker='^', label='Center station')
ax1.scatter(x_array, y_array, s=200, c='blue', marker='v', label='Outer stations')
circle = plt.Circle((0, 0), r, fill=False, linestyle='--', color='gray')
ax1.add_artist(circle)

# 波の到来方向を矢印で表示
for angle in [0, np.pi/3, 2*np.pi/3]:
    ax1.arrow(-r*1.5*np.cos(angle), -r*1.5*np.sin(angle), 
              r*0.8*np.cos(angle), r*0.8*np.sin(angle),
              head_width=10, head_length=10, fc='orange', ec='orange', alpha=0.7)

ax1.set_xlim(-100, 100)
ax1.set_ylim(-100, 100)
ax1.set_aspect('equal')
ax1.set_xlabel('X [m]')
ax1.set_ylabel('Y [m]')
ax1.set_title('Microtremor Array Configuration')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 探査深度と周波数の関係
frequencies = np.logspace(-1, 1.5, 50)
vs = 200  # 仮定したS波速度 [m/s]
wavelengths = vs / frequencies
depth_min = wavelengths / 3
depth_max = wavelengths / 2

ax2.fill_between(frequencies, depth_min, depth_max, alpha=0.3, label='Investigation depth range')
ax2.plot(frequencies, depth_min, 'b--', label=r'$\lambda/3$')
ax2.plot(frequencies, depth_max, 'r--', label=r'$\lambda/2$')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Depth [m]')
ax2.set_title('Frequency vs Investigation Depth')
ax2.legend()
ax2.grid(True, which='both', alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

## 1. 微動アレイ観測とは

### 1.1 微動アレイ観測の目的

微動アレイ観測は、地盤の常時微動を複数の地震計で同時観測し、表面波の分散特性から地下のS波速度構造を推定する手法です。

**主な目的：**
1. **波の振動数ごとの速度（位相速度）を知る**
2. **地盤のS波速度分布を推定する**

### 1.2 なぜ微動から地下構造がわかるのか？

- 微動の主成分は**表面波（レイリー波）**
- 表面波の速度は**周波数によって変化**（分散性）
- **低周波数**の波ほど**深くまで到達**
- 各周波数の速度から、対応する深さの地盤の硬さがわかる

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import signal, special
from scipy.optimize import curve_fit
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# プロット設定
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# 微動アレイ観測解析 チュートリアル

このノートブックでは、微動アレイ観測の基礎から実践までを学びます。

## 目次
1. [微動アレイ観測とは](#1-微動アレイ観測とは)
2. [波動の基礎理論](#2-波動の基礎理論)
3. [SPAC法の原理と実装](#3-SPAC法の原理と実装)
4. [F-K法の原理と実装](#4-F-K法の原理と実装)
5. [実データ解析](#5-実データ解析)
6. [地下構造推定](#6-地下構造推定)